# OpenCV Face Detection Webcam

In this notebook, opencv face and body detection will be applied to webcam images.

To run all cells in this notebook a webcam is required and the Delivery Tracker Django project must be running.

To stop running the camera, interrupt the kernel.


### Step 1: Enter the serial number found on the back of your PYNQ-Z2 Board and the URL of your server.

In [6]:
SERIAL_NUMBER = "911396a7-de99-49e0-b23d-643f48f08348"
URL = "http://192.168.1.143:9595"

### Step 2: Load the overlay

In [7]:
# load the overlay 
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
base = BaseOverlay("base.bit")

### Step 3: Initialize HDMI Out

In [8]:
# monitor configuration
Mode = VideoMode(1280,720,24)
hdmi_out = base.video.hdmi_out
hdmi_out.configure(Mode,PIXEL_BGR)
hdmi_out.start()

### Step 4: Initailize Webcam

In [9]:
import os
os.environ["OPENCV_LOG_LEVEL"]="SILENT"

import cv2
#initialize camera from openCV
videoIn = cv2.VideoCapture(0)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

print("Capture device is open: " + str(videoIn.isOpened()))

Capture device is open: True


### Step 5: Continuously Monitor

In [10]:
import numpy as np
import requests
import sys
import time

face_cascade = cv2.CascadeClassifier('data/haarcascade_frontalface_default.xml')
body_cascade = cv2.CascadeClassifier('data/haarcascade_upperbody.xml')
 
if (body_cascade.empty()):
    print("Failed to load cascade from file.")

try:
    while True:   
        # read next image
        ret, frame_vga = videoIn.read()
        outframe = hdmi_out.newframe()
        outframe[:] = frame_vga
        gray_image = cv2.cvtColor(outframe, cv2.COLOR_BGR2GRAY)
        # detect multiscale returns x, y, w, h
        faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.4, minNeighbors=5, minSize=(75, 75))
        upper_bodies = body_cascade.detectMultiScale(gray_image, 1.08, 1, minSize=(200, 200))

        if (ret):    
            if(len(faces)>0 or len(upper_bodies)>0):
                frameList = outframe.tolist()
                # Add your Django hosting server's IP
                link = URL + "/uploadFrame/"
                # Add your device ID
                data = {'param': SERIAL_NUMBER, 'arr': frameList}
                response = requests.post(link, json=data)
                time.sleep(3)
        else:
            # video input disconnected
            videoIn.release()
            hdmi_out.stop()
            del hdmi_out
            sys.exit()

except KeyboardInterrupt:
    videoIn.release()
    hdmi_out.stop()
    del hdmi_out
